**DATASET CLASS**


In [ ]:
pip install torch

c:\Users\grozz\AppData\Local\Programs\Microsoft VS Code


In [32]:
import pandas as pd
import os
import numpy as np
#sensor_df = pd.read_csv("P001.csv")

#labels_df = pd.read_csv("annotation-label-dictionary.csv")

labels_df = pd.read_csv("annotation-label-dictionary.csv")

# Pick one label scheme:
chosen_column = "label:Willetts2018"

# Map the string labels to integer classes
label_mapping = {label: i for i, label in enumerate(labels_df[chosen_column].unique())}
print("Label Mapping:", label_mapping)

subject_labels = labels_df[chosen_column].map(label_mapping).values


sensor_folder = "pids"
select_pids = ["P001", "P002", "P003", "P004", "P005"]


# Convert labels to integers
label_mapping = {label: i for i, label in enumerate(labels_df[chosen_column].unique())}
mapped_labels = labels_df[chosen_column].map(label_mapping).values

# Now split up the labels across subjects in `select_pids`
data_list = []
label_list = []

offset = 0  # position in mapped_labels

for pid in select_pids:
    file_path = os.path.join(sensor_folder, f"{pid}.csv")
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        sensor_data = df[["x", "y", "z"]].values
        sensor_data = (sensor_data - sensor_data.mean(axis=0)) / sensor_data.std(axis=0)

        n_samples = len(sensor_data)
        labels = mapped_labels[offset:offset + n_samples]
        offset += n_samples

        data_list.append(sensor_data)
        label_list.append(labels)

Label Mapping: {'sleep': 0, 'sit-stand': 1, 'mixed': 2, 'walking': 3, 'vehicle': 4, 'bicycling': 5}


C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\4070161790.py:38: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\4070161790.py:38: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\4070161790.py:38: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\4070161790.py:38: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\4070161790.py:38: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [44]:
label_df = pd.read_csv("annotation-label-dictionary.csv")
chosen_column = "label:Willetts2018"
annotation_to_label = dict(zip(label_df["annotation"], label_df[chosen_column]))
label_mapping = {label: i for i, label in enumerate(label_df[chosen_column].unique())}


for pid in select_pids:
    file_path = os.path.join(sensor_folder, f"{pid}.csv")
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Normalize sensor data
        sensor_data = df[["x", "y", "z"]].values
        sensor_data = (sensor_data - sensor_data.mean(axis=0)) / sensor_data.std(axis=0)
        data_list.append(sensor_data)

        # Map annotation strings to class labels
        annotations = df["annotation"].map(annotation_to_label)
        labels = annotations.map(label_mapping)
        labels = labels.fillna(-1).astype(int).values  # -1 for unknown/missing labels
        label_list.append(labels)
        
valid_indices = labels != -1
sensor_data = sensor_data[valid_indices]
labels = labels[valid_indices]

C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\693305920.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\693305920.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\693305920.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\693305920.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\693305920.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [83]:
from torch.utils.data import Dataset
import torch
import numpy as np



class Capture24Dataset(Dataset):
    def __init__(self, data_list, label_list, window_size=256, stride=128):
        self.X = []
        self.y = []

        for data, labels in zip(data_list, label_list):
            for i in range(0, len(data) - window_size, stride):
                window = data[i:i+window_size]
                label_window = labels[i:i+window_size]
                if len(label_window) == 0 or np.all(label_window == -1):
                    continue  
                if np.any(np.array(label_window) < 0):
                    continue  
                label = np.bincount(label_window).argmax()
                self.X.append(window)
                self.y.append(label)

        self.X = torch.tensor(np.array(self.X), dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [54]:
dataset = Capture24Dataset(data_list, label_list, window_size=256, stride=128)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
class ActivityModel(nn.Module):
    def __init__(self, input_channels=6, num_classes=6):
        super(ActivityModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.lstm = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):  # [B, T, C]
        x = x.permute(0, 2, 1)  # → [B, C, T]
        x = self.cnn(x)         # → [B, 32, T']
        x = x.permute(0, 2, 1)  # → [B, T', 32]
        _, (h_n, _) = self.lstm(x)
        return self.classifier(h_n.squeeze(0))

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ActivityModel(input_channels=3).to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
 
for epoch in range(10): 
    model.train() 
    total_loss = 0 
    for x_batch, y_batch in train_loader: 
        x_batch, y_batch = x_batch.to(device), y_batch.to(device) 
        optimizer.zero_grad() 
        output = model(x_batch) 
        loss = criterion(output, y_batch) 
        loss.backward() 
        optimizer.step() 
        total_loss += loss.item() 
     
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}") 

Epoch 1, Loss: 0.9630
Epoch 2, Loss: 0.7489
Epoch 3, Loss: 0.6694
Epoch 4, Loss: 0.6093
Epoch 5, Loss: 0.5541
Epoch 6, Loss: 0.5243
Epoch 7, Loss: 0.4989
Epoch 8, Loss: 0.4782
Epoch 9, Loss: 0.4644
Epoch 10, Loss: 0.4529


Evaluation

In [58]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = y_batch.numpy()

        predictions.extend(preds)
        actuals.extend(labels)

In [61]:
import pandas as pd

inv_label_mapping = {v: k for k, v in label_mapping.items()}  # To decode labels

results_df = pd.DataFrame({
    "Actual": [inv_label_mapping[a] for a in actuals],
    "Predicted": [inv_label_mapping[p] for p in predictions]
})

print(results_df.head(10))  # Show first 10

      Actual  Predicted
0    walking    walking
1      sleep      sleep
2      sleep      sleep
3    vehicle    vehicle
4  bicycling  bicycling
5      mixed      mixed
6      sleep      sleep
7  sit-stand  sit-stand
8    walking  sit-stand
9  sit-stand  sit-stand


In [62]:
#Accuracy
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8340


Test Model with New Data

In [101]:


label_mapping = {label: i for i, label in enumerate(labels_df[chosen_column].unique())}
print("Label Mapping:", label_mapping)

subject_labels = labels_df[chosen_column].map(label_mapping).values


sensor_folder = "pids"
select_pids = ["P006", "P007", "P008", "P009", "P010"]


# Convert labels to integers
label_mapping = {label: i for i, label in enumerate(labels_df[chosen_column].unique())}
mapped_labels = labels_df[chosen_column].map(label_mapping).values

# Now split up the labels across subjects in `select_pids`
data_list = []
label_list = []

offset = 0  # position in mapped_labels

for pid in select_pids:
    file_path = os.path.join(sensor_folder, f"{pid}.csv")
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        sensor_data = df[["x", "y", "z"]].values
        sensor_data = (sensor_data - sensor_data.mean(axis=0)) / sensor_data.std(axis=0)

        n_samples = len(sensor_data)
        labels = mapped_labels[offset:offset + n_samples]
        offset += n_samples

        data_list.append(sensor_data)
        label_list.append(labels)

Label Mapping: {'sleep': 0, 'sit-stand': 1, 'mixed': 2, 'walking': 3, 'vehicle': 4, 'bicycling': 5}


C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\1044544365.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\1044544365.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\1044544365.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\1044544365.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\1044544365.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [103]:
label_list = []
data_list = []
label_df = pd.read_csv("annotation-label-dictionary.csv")   
for pid in select_pids:
    file_path = os.path.join(sensor_folder, f"{pid}.csv")
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        sensor_data = df[["x", "y", "z"]].values
        sensor_data = (sensor_data - sensor_data.mean(axis=0)) / sensor_data.std(axis=0)

        # Map annotation strings to class labels
        annotations = df["annotation"].map(annotation_to_label)
        labels = annotations.map(label_mapping)
        labels = labels.fillna(-1).astype(int).values

        # Filter out invalid labels BEFORE appending
        valid_indices = labels != -1
        sensor_data = sensor_data[valid_indices]
        labels = labels[valid_indices]

        if len(sensor_data) >= 256:
            data_list.append(sensor_data)
            label_list.append(labels)
        else:
            print(f"{pid} skipped — not enough valid data")

C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\942780500.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\942780500.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\942780500.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\942780500.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\grozz\AppData\Local\Temp\ipykernel_25356\942780500.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [104]:
dataset = Capture24Dataset(data_list, label_list, window_size=256, stride=128)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ActivityModel(input_channels=3).to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
 
for epoch in range(10): 
    model.train() 
    total_loss = 0 
    for x_batch, y_batch in train_loader: 
        x_batch, y_batch = x_batch.to(device), y_batch.to(device) 
        optimizer.zero_grad() 
        output = model(x_batch) 
        loss = criterion(output, y_batch) 
        loss.backward() 
        optimizer.step() 
        total_loss += loss.item() 
     
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}") 

Epoch 1, Loss: 0.7935
Epoch 2, Loss: 0.5448
Epoch 3, Loss: 0.4676


In [ ]:
P